# Volvo Trucks Analytics

Imports and global variables go here

In [1]:
import pandas as pd
import numpy as np

truck1csv = "../data/trucks/truck1.csv"
truck2csv = "../data/trucks/truck2.csv"

Define all cleaning methods here

In [2]:
def readCsv (truck):
    return pd.read_csv(truck, header=[0])
    
def cullUtcCols (truckData):
    noUTC = truckData.drop(["UTC hour", "UTC minute", "UTC second", "UTC month", "UTC day", "UTC year"], axis = 1)
    return noUTC

Cleaning of Truck 1 Data starts here

In [3]:
truck1data = readCsv(truck1csv)
truck1data = cullUtcCols(truck1data)

# After cleaning, check the shape of the dataframe
truck1data.shape

(1216689, 45)

Cleaning of Truck 2 Data starts here

In [4]:
truck2data = readCsv(truck2csv)
truck2data = cullUtcCols(truck2data)

# After cleaning, check the shape of the dataframe
truck2data.shape

(1686323, 40)